In [1]:
import copy

In [2]:
genesis = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.00},
    "Trader":{"r_1":100,"r_2":100,"s":0},
    "LP":{"r_1":0,"r_2":0,"s":100}
    }

In [3]:
def swapToAsset2(state,inputs):
    agent = inputs[0]
    dA1 = inputs[1]
    feeFactor = (1-state["AMM"]["fee"])
    dA2 = state["AMM"]["r_2"]/(state["AMM"]["r_1"]+dA1*feeFactor)*dA1*feeFactor
    if dA1>0 and state[agent]["r_1"]-dA1 >= 0 :
        state["AMM"]["r_1"]+=dA1
        state[agent]["r_1"]-=dA1
        state["AMM"]["r_2"]-=dA2
        state[agent]["r_2"]+=dA2 
        
def swapToAsset1(state,inputs):
    agent = inputs[0]
    dA2 = inputs[1]
    feeFactor = (1-state["AMM"]["fee"])
    dA1 = state["AMM"]["r_1"]/(state["AMM"]["r_2"]+dA2*feeFactor)*dA2*feeFactor
    if dA2>0 and state[agent]["r_2"]-dA2 >= 0 :
        state["AMM"]["r_2"]+=dA2
        state[agent]["r_2"]-=dA2
        state["AMM"]["r_1"]-=dA1
        state[agent]["r_1"]+=dA1 
        
def removeLiquidity(state,inputs):
    dS = inputs[1]
    agent = inputs[0]
    if dS > 0 and state[agent]["s"]-dS>=0 and state["AMM"]["s"]-dS>=0:
        DR = (1-dS/state["AMM"]["s"])
        r_1=state["AMM"]["r_1"]
        r_2=state["AMM"]["r_2"]
        state[agent]["s"]-=dS
        state["AMM"]["r_1"]=r_1*DR
        state["AMM"]["r_2"]=r_2*DR
        state[agent]["r_1"]+=r_1-state["AMM"]["r_1"]
        state[agent]["r_2"]+=r_2-state["AMM"]["r_2"]
        state["AMM"]["s"]-=dS
        

def nice_print(self):
    if type(self)==float:
        return  round(self,3)
    if type(self)==int:
        return  self
    if type(self)==list:
        return  list(map(lambda l: print(l),self))
    if type(self)==dict:
        return  dict(map(lambda kv: (kv[0], print(kv[1])),self.items()))
    
def evolve(state, actionStack):
    history = [copy.deepcopy(state)]
    for action in actionStack:
        action[0](state,action[1])
        history.append(copy.deepcopy(state))
    return history 

In [4]:
actionList = [
        [ swapToAsset2 , [  "Trader" , 50 ]],
        [ swapToAsset1 , [  "Trader" , 25 ]],
]

In [5]:
evolve(genesis,actionList)
genesis

{'AMM': {'r_1': 109.0909090909091,
  'r_2': 91.66666666666667,
  's': 100,
  'fee': 0.0},
 'Trader': {'r_1': 90.9090909090909, 'r_2': 108.33333333333331, 's': 0},
 'LP': {'r_1': 0, 'r_2': 0, 's': 100}}

In [ ]:
def check_genesis_block(genesis, actionList):
    for item in actionList:
        if item[1][1] < 0:
            return False
    return True